In [2]:
from langchain_community.document_loaders import PyPDFLoader
import os
import re

files = os.listdir('transcripts/')
for file in files[0:1]:
    loader = PyPDFLoader('transcripts/'+file)
    pages = loader.load_and_split()

    for page in pages:
        lines = page.page_content.split('\n')
        lines_trimmed = list(map(str.strip, lines))
        cleaned_text = list(map(lambda s: re.sub(r'\s+\d{1,2}\s+$', '', s), lines))[3:-2]
        page.page_content = '\n'.join(cleaned_text)
    
    with open('transcripts/'+file[:-3]+'txt', 'w') as f:
            for page in pages:
                f.write(page.page_content)
                f.write('\n')

In [6]:
def pagechunker(chunk_size, overlap_size, pages): # takes a list of Documents (pages)
    start_page = 0
    total_pages = len(pages)
    chunks = []
    while start_page < total_pages:
        chunk_text = ''
        for page in pages[start_page:min(start_page+chunk_size, total_pages-1)]:
            chunk_text+=(page.page_content)
        start_page = start_page+overlap_size
        chunks.append(chunk_text)
        
    return chunks

chunks = pagechunker(40, 10, pages)